In [1]:
import xarray as xr
import numpy as np
import cmocean as cm
import os
import pandas as pd
from matplotlib import cm as cmap
%matplotlib inline

In [2]:
data=pd.read_csv('/g/data/v45/jm5970/AVISO_dataframe/trackeddy_V2.0_global_eddies_V2.csv')
data['time']=pd.to_datetime(data['time'])

In [3]:
bins_x = np.hstack((np.arange(0.1,359.9,1),359.95))
bins_y = np.arange(-70,70)

In [4]:
data_fields=data[['time','neddy','eddy_amplitude','area_eddy','maxima_loc_y','maxima_loc_x']].set_index('time').sort_index()

In [5]:
data_fields['diameter'] = ( np.sqrt(data_fields[['area_eddy']]/(np.pi))*2 )/ 1000

In [6]:
data_fields['binned_x'] = pd.cut(data_fields['maxima_loc_x'], bins_x)
data_fields['binned_y'] = pd.cut(data_fields['maxima_loc_y'], bins_y)

In [7]:
neddy_count = data_fields.pivot_table(index="binned_x", columns="binned_y", values="neddy",aggfunc=('count')).fillna(0)

In [8]:
mean_amp = data_fields.pivot_table(index="binned_x", columns="binned_y", values="eddy_amplitude",aggfunc=('mean')).fillna(0)

In [9]:
pos_eddy = data_fields.where(data_fields['eddy_amplitude']>0).pivot_table(index="binned_x", columns="binned_y", values="eddy_amplitude",aggfunc=('mean')).fillna(0)
neg_eddy = data_fields.where(data_fields['eddy_amplitude']<0).pivot_table(index="binned_x", columns="binned_y", values="eddy_amplitude",aggfunc=('mean')).fillna(0)

In [10]:
diameter = data_fields.pivot_table(index="binned_x", columns="binned_y", values="diameter",aggfunc=('mean')).fillna(0)

In [11]:
neddy_stats = np.zeros((len(bins_y),len(bins_x)))*np.nan
ampeddy_stats = np.zeros((len(bins_y),len(bins_x)))*np.nan
diameter_stats = np.zeros((len(bins_y),len(bins_x)))*np.nan

pos_eddy_stats = np.zeros((len(bins_y),len(bins_x)))*np.nan
neg_eddy_stats = np.zeros((len(bins_y),len(bins_x)))*np.nan

neddy_stats[:-1,:-1] = neddy_count.T
ampeddy_stats[:-1,:-1] = mean_amp.T
diameter_stats[:-1,:-1] = diameter.T

pos_eddy_stats[:-1,:-1] = pos_eddy.T
neg_eddy_stats[:-1,:-1] = neg_eddy.T


eddy_climat = xr.Dataset(
    {
        "eddy_count": (["lat", "lon"], neddy_stats),
        "eddy_amp": (["lat", "lon"], ampeddy_stats),
        "eddy_diameter": (["lat", "lon"], diameter_stats),
        "pos_eddy_amp": (["lat", "lon"], pos_eddy_stats),
        "neg_eddy_amp": (["lat", "lon"], neg_eddy_stats),
    },
    coords={
        "lon": (["lon"], bins_x[:]),
        "lat": (["lat"], bins_y[:])
    },
)

In [12]:
eddy_climat = eddy_climat.interpolate_na(dim="lon", method="linear", fill_value="extrapolate")

In [13]:
eddy_climat.to_netcdf('../datasets/gridded_TrackEddy_stats.nc')